<a href="https://colab.research.google.com/github/Orasz/CNN4COVID19/blob/main/customUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/albumentations-team/albumentations.git

  Cloning https://github.com/albumentations-team/albumentations.git to /tmp/pip-req-build-5ihtlvvs
  Running command git clone -q https://github.com/albumentations-team/albumentations.git /tmp/pip-req-build-5ihtlvvs
  Created wheel for albumentations: filename=albumentations-0.5.2-cp37-none-any.whl size=86173 sha256=fdd0148e9c9cb5f680ec28e4f5bffb84e3b49a8be20288d8d3410fe3d32100fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-whpksdva/wheels/e2/85/3e/2a40fac5cc1f43ced656603bb2fca1327b30ec7de1b1b66517
Successfully built albumentations


In [8]:

import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim


#DATASET


class QaTaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.masks = os.listdir(mask_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.masks[index])
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

In [3]:
#UTILS
import torch
import torchvision
from torch.utils.data import DataLoader

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = QaTaDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = QaTaDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

In [4]:
#MODEL
import torch.nn as nn
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)


In [29]:
#TRAINING
# Hyperparameters etc.
LEARNING_RATE = 1e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 50
NUM_WORKERS = 2
IMAGE_HEIGHT = 224  
IMAGE_WIDTH = 224 
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "/content/drive/MyDrive/UNET/train_img/"
TRAIN_MASK_DIR = "/content/drive/MyDrive/UNET/train_masks/"
VAL_IMG_DIR = "/content/drive/MyDrive/UNET/val_img/"
VAL_MASK_DIR = "/content/drive/MyDrive/UNET/val_masks/"

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            #A.centerCrop(224),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            #A.centerCrop(224),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)


    check_accuracy(val_loader, model, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)
        check_accuracy(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder="/content/drive/MyDrive/UNET/immagini_salvate_unet_10epoche/", device=DEVICE
        )



In [30]:
main()





 11%|█         | 16/148 [00:06<00:49,  2.68it/s, loss=0.343]



 11%|█▏        | 17/148 [00:06<00:48,  2.70it/s, loss=0.343]



 11%|█▏        | 17/148 [00:06<00:48,  2.70it/s, loss=0.352]



 12%|█▏        | 18/148 [00:06<00:48,  2.68it/s, loss=0.352]



 12%|█▏        | 18/148 [00:07<00:48,  2.68it/s, loss=0.292]



 13%|█▎        | 19/148 [00:07<00:48,  2.67it/s, loss=0.292]



 13%|█▎        | 19/148 [00:07<00:48,  2.67it/s, loss=0.334]



 14%|█▎        | 20/148 [00:07<00:47,  2.69it/s, loss=0.334]



 14%|█▎        | 20/148 [00:08<00:47,  2.69it/s, loss=0.324]



 14%|█▍        | 21/148 [00:08<00:47,  2.67it/s, loss=0.324]



 14%|█▍        | 21/148 [00:08<00:47,  2.67it/s, loss=0.337]



 15%|█▍        | 22/148 [00:08<00:47,  2.67it/s, loss=0.337]



 15%|█▍        | 22/148 [00:08<00:47,  2.67it/s, loss=0.341]



 16%|█▌        | 23/148 [00:08<00:46,  2.68it/s, loss=0.341]



 16%|█▌        | 23/148 [00:09<00:46,  2.68it/s, loss=0.304]



 16%|█▌        | 24/148 [00:09<00:46

Got 25261590/29654016 with acc 85.19
Dice score: 0.184907466173172






  0%|          | 0/148 [00:00<?, ?it/s]



  0%|          | 0/148 [00:00<?, ?it/s, loss=0.333]



  1%|          | 1/148 [00:00<01:30,  1.63it/s, loss=0.333]



  1%|          | 1/148 [00:00<01:30,  1.63it/s, loss=0.338]



  1%|▏         | 2/148 [00:00<01:18,  1.86it/s, loss=0.338]



  1%|▏         | 2/148 [00:01<01:18,  1.86it/s, loss=0.346]



  2%|▏         | 3/148 [00:01<01:10,  2.04it/s, loss=0.346]



  2%|▏         | 3/148 [00:01<01:10,  2.04it/s, loss=0.29] 



  3%|▎         | 4/148 [00:01<01:05,  2.20it/s, loss=0.29]



  3%|▎         | 4/148 [00:02<01:05,  2.20it/s, loss=0.305]



  3%|▎         | 5/148 [00:02<01:01,  2.34it/s, loss=0.305]



  3%|▎         | 5/148 [00:02<01:01,  2.34it/s, loss=0.312]



  4%|▍         | 6/148 [00:02<00:58,  2.44it/s, loss=0.312]



  4%|▍         | 6/148 [00:02<00:58,  2.44it/s, loss=0.29] 



  5%|▍         | 7/148 [00:02<00:56,  2.51it/s, loss=0.29]



  5%|▍         | 7/148 [00:03<00:56,  2.51it/s, loss=0.319]



  5%|▌         | 8

KeyboardInterrupt: ignored